In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44659
2,Huelva,Confirmados PDIA 14 días,770
3,Huelva,Tasa PDIA 14 días,"150,04774246351113"
4,Huelva,Confirmados PDIA 7 días,257
5,Huelva,Tasa PDIA 7 dias,"50,08086988717189"
6,Huelva,Total Confirmados,44858
7,Huelva,Curados,40892
8,Huelva,Fallecidos,406


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44659.0


/tmp/ipykernel_5471/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12381.0


/tmp/ipykernel_5471/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_5471/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_5471/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_5471/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2766 personas en los últimos 7 días 

Un positivo PCR cada 888 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44659.0,257.0,770.0,513170.0,50.080870,150.047742,19.0
Huelva-Costa,26392.0,166.0,476.0,289548.0,57.330736,164.394159,12.0
Condado-Campiña,13898.0,61.0,210.0,156231.0,39.044748,134.416345,3.0
Isla Cristina,3066.0,25.0,66.0,21393.0,116.860655,308.512130,3.0
Sierra de Huelva-Andévalo Central,3951.0,25.0,73.0,67391.0,37.096942,108.323070,3.0
Alosno,229.0,3.0,4.0,3933.0,76.277651,101.703534,2.0
Aracena,667.0,5.0,14.0,8255.0,60.569352,169.594185,2.0
Ayamonte,2002.0,17.0,42.0,21104.0,80.553450,199.014405,2.0
Cartaya,1880.0,28.0,70.0,20083.0,139.421401,348.553503,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,31.0,1.0,5.0,396.0,252.525253,1262.626263,0.0
San Silvestre de Guzmán,24.0,3.0,7.0,614.0,488.599349,1140.065147,0.0
San Bartolomé de la Torre,376.0,4.0,21.0,3761.0,106.354693,558.362138,0.0
Villablanca,281.0,9.0,14.0,2885.0,311.958406,485.268631,0.0
Cerro de Andévalo (El),195.0,4.0,10.0,2327.0,171.895144,429.737860,0.0
Higuera de la Sierra,62.0,4.0,5.0,1291.0,309.837335,387.296669,0.0
Cartaya,1880.0,28.0,70.0,20083.0,139.421401,348.553503,2.0
Lucena del Puerto,362.0,1.0,11.0,3261.0,30.665440,337.319841,0.0
Isla Cristina,3066.0,25.0,66.0,21393.0,116.860655,308.512130,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aljaraque,1608.0,1.0,18.0,21474.0,4.656794,83.822297,0.0,0.055556
Palma del Condado (La),1412.0,1.0,18.0,10801.0,9.258402,166.651236,0.0,0.055556
Lucena del Puerto,362.0,1.0,11.0,3261.0,30.665440,337.319841,0.0,0.090909
Beas,358.0,1.0,8.0,4341.0,23.036167,184.289334,0.0,0.125000
San Bartolomé de la Torre,376.0,4.0,21.0,3761.0,106.354693,558.362138,0.0,0.190476
Gibraleón,923.0,4.0,20.0,12737.0,31.404569,157.022847,0.0,0.200000
Cañaveral de León,31.0,1.0,5.0,396.0,252.525253,1262.626263,0.0,0.200000
Valverde del Camino,831.0,2.0,8.0,12750.0,15.686275,62.745098,0.0,0.250000
Punta Umbría,1250.0,5.0,20.0,15355.0,32.562683,130.250733,0.0,0.250000
